In [1]:
from pathlib import Path
from metasmith.agents.presets import Agent, AGENT_SETUP_COMPLETE
from metasmith.models.libraries import *
from metasmith.models.remote import *

from local.constants import WORKSPACE_ROOT

In [2]:
agent = Agent(
    setup_commands=[
        f'echo "{AGENT_SETUP_COMPLETE}"',
    ],
    cleanup_commands=[],
    home=Source.FromLocal(WORKSPACE_ROOT/"main/local_mock/cache/local_home"),
)

# agent.Deploy()

In [3]:
trlib = TransformInstanceLibrary.Load("./transforms/simple_1")
for path, name, tr in trlib.IterateTransforms():
    print(name, tr)

transforms::transform TransformInstance(protocol=<function protocol at 0x7fdc6acc39c0>, model={{"data":"software container"}-{"format":"OCI"}-{"provides":"diamond"}},{{"data":"Amino acid sequence"}-{"format":"FASTA"}},{{"data":"database reference"}-{"format":".dmnd"}}->{{"data":"Protein features"}-{"format":"CSV"}}, output_signature={(D:{"data":"Protein features"}-{"format":"CSV"}): PosixPath('annotations.csv')}, name='diamond')
transforms::transform TransformInstance(protocol=<function protocol at 0x7fdc6acc3a60>, model={{"data":"software container"}-{"format":"OCI"}-{"provides":"pprodigal"}},{{"data":"DNA sequence"}-{"format":"FASTA"}}->{{"data":"Amino acid sequence"}-{"format":"FASTA"}}, output_signature={(D:{"data":"Amino acid sequence"}-{"format":"FASTA"}): PosixPath('orfs.faa')}, name='pprodigal')
